In [ ]:

!pip uninstall -q tensorflow==2.16.1



Proceed (Y/n)? 

In [ ]:
!pip install -q tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 987.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 67.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 56.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 73.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 25.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.15.1 requires tensorflow<2.16,>=2.15, but you have tensorflow 2.16.1 which is incompatible.


In [ ]:
import tensorflow as tf
print(tf.__version__)  # Should print 2.16.1


2.15.0


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 784).astype('float32') / 255.0
x_test = x_test.reshape(-1, 784).astype('float32') / 255.0


def create_model():
    model = models.Sequential()
    model.add(layers.Input(shape=(784,)))
    model.add(layers.Dense(50, activation='relu', name='dense'))
    model.add(layers.Dense(20, activation='relu', name='dense_1'))
    model.add(layers.Dense(10, activation='relu', name='dense_2'))
    model.add(layers.Dense(10, activation='relu', name='dense_3'))
    return model

model = create_model()

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(),
    metrics=["accuracy"],
)

model.fit(x_train, y_train, batch_size=128, epochs=20, validation_split=0.1)
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")


layer_weights = {}
for layer in model.layers:
    if isinstance(layer, layers.Dense):
        weights, biases = layer.get_weights()
        layer_weights[layer.name] = {'weights': weights, 'biases': biases}
        print(f"Layer name: {layer.name}")
        print(f"Weights shape: {weights.shape}")
        print(f"Biases shape: {biases.shape}")

# Representative dataset generator for quantization
def representative_data_gen():
    for input_value in tf.data.Dataset.from_tensor_slices(x_train).batch(1).take(100):
        yield [tf.dtypes.cast(input_value, tf.float32)]

# Create a concrete function from the Keras model
@tf.function(input_signature=[tf.TensorSpec(shape=[None, 784], dtype=tf.float32)])
def model_func(inputs):
    return model(inputs)

concrete_func = model_func.get_concrete_function()

# Convert the model to a TensorFlow Lite model with int8 quantization
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
quantized_tflite_model = converter.convert()

with open("quantized_model_scaled.tflite", "wb") as f:
    f.write(quantized_tflite_model)

# Function to save the weights and biases of the quantized model
def save_quantized_weights_and_biases(tflite_model, filename, layer_weights):
    interpreter = tf.lite.Interpreter(model_content=tflite_model)
    interpreter.allocate_tensors()

    tensor_details = interpreter.get_tensor_details()

    print(f"Number of tensors in the quantized model: {len(tensor_details)}")
    print(f"Number of layers in Keras model: {len(layer_weights)}")

    with open(filename + '_quantized_params.txt', "w") as file:
        for i, tensor_detail in enumerate(tensor_details):
            tensor_name = tensor_detail['name']
            try:
                tensor = interpreter.get_tensor(tensor_detail['index'])
            except ValueError as e:
                print(f"Skipping tensor {tensor_name} due to error: {e}")
                continue

            header = f"{tensor_name} (index: {i})"
            if "BiasAdd/ReadVariableOp/resource" in tensor_name:
                layer_name = tensor_name.split('/')[1]
                header += f", corresponds to layer: {layer_name} biases"
            elif "MatMul" in tensor_name and "BiasAdd" not in tensor_name:
                layer_name = tensor_name.split('/')[1]
                header += f", corresponds to layer: {layer_name} weights"

            file.write(f"# {header}\n")
            np.savetxt(file, tensor)

# Save the weights and biases
save_quantized_weights_and_biases(quantized_tflite_model, "quantized_model_scaled", layer_weights)


Epoch 1/20
422/422 [==============================] - 7s 11ms/step - loss: 0.8932 - accuracy: 0.7233 - val_loss: 0.2869 - val_accuracy: 0.9217
Epoch 2/20
422/422 [==============================] - 2s 4ms/step - loss: 0.3041 - accuracy: 0.9132 - val_loss: 0.2304 - val_accuracy: 0.9338
Epoch 3/20
422/422 [==============================] - 2s 5ms/step - loss: 0.2386 - accuracy: 0.9304 - val_loss: 0.1747 - val_accuracy: 0.9497
Epoch 4/20
422/422 [==============================] - 3s 7ms/step - loss: 0.2038 - accuracy: 0.9407 - val_loss: 0.1527 - val_accuracy: 0.9590
Epoch 5/20
422/422 [==============================] - 2s 6ms/step - loss: 0.1772 - accuracy: 0.9478 - val_loss: 0.1280 - val_accuracy: 0.9615
Epoch 6/20
422/422 [==============================] - 2s 4ms/step - loss: 0.1586 - accuracy: 0.9527 - val_loss: 0.1304 - val_accuracy: 0.9610
Epoch 7/20
422/422 [==============================] - 2s 5ms/step - loss: 0.1462 - accuracy: 0.9566 - val_loss: 0.1184 - val_accuracy: 0.9657
Epoch

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 784).astype('float32') / 255.0
x_test = x_test.reshape(-1, 784).astype('float32') / 255.0


def create_model():
    model = models.Sequential()
    model.add(layers.Input(shape=(784,)))
    model.add(layers.Dense(50, activation='relu', name='dense'))
    model.add(layers.Dense(20, activation='relu', name='dense_1'))
    model.add(layers.Dense(10, activation='relu', name='dense_2'))
    model.add(layers.Dense(10, activation='relu', name='dense_3'))
    return model

model = create_model()

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(),
    metrics=["accuracy"],
)

model.fit(x_train, y_train, batch_size=128, epochs=20, validation_split=0.1)
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")

layer_weights = {}
for layer in model.layers:
    if isinstance(layer, layers.Dense):
        weights, biases = layer.get_weights()
        layer_weights[layer.name] = {'weights': weights, 'biases': biases}
        print(f"Layer name: {layer.name}")
        print(f"Weights shape: {weights.shape}")
        print(f"Biases shape: {biases.shape}")

# Representative dataset generator for quantization
def representative_data_gen():
    for input_value in tf.data.Dataset.from_tensor_slices(x_train).batch(1).take(100):
        yield [tf.dtypes.cast(input_value, tf.float32)]

# Create a concrete function from the Keras model
@tf.function(input_signature=[tf.TensorSpec(shape=[None, 784], dtype=tf.float32)])
def model_func(inputs):
    return model(inputs)

concrete_func = model_func.get_concrete_function()

# Convert the model to a TensorFlow Lite model with int8 quantization
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
quantized_tflite_model = converter.convert()


with open("quantized_model_int8.tflite", "wb") as f:
    f.write(quantized_tflite_model)

# Function to save the weights and biases of the quantized model
def save_quantized_weights_and_biases(tflite_model, filename):
    interpreter = tf.lite.Interpreter(model_content=tflite_model)
    interpreter.allocate_tensors()

    tensor_details = interpreter.get_tensor_details()

    with open(filename + '_quantized_params.txt', "w") as file:
        for tensor_detail in tensor_details:
            tensor_name = tensor_detail['name']
            if "BiasAdd" in tensor_name or "MatMul" in tensor_name:
                try:
                    tensor = interpreter.get_tensor(tensor_detail['index'])
                    # Check if tensor has valid data
                    if tensor is not None:
                        header = f"{tensor_name} (index: {tensor_detail['index']})"
                        print(f"Tensor length is: {len(tensor)}")
                        if "BiasAdd" in tensor_name:
                            layer_name = tensor_name.split('/')[1]
                            header += f", corresponds to layer: {layer_name} biases"
                        elif "MatMul" in tensor_name:
                            layer_name = tensor_name.split('/')[1]
                            header += f", corresponds to layer: {layer_name} weights"

                        file.write(f"# {header}\n")
                        np.savetxt(file, tensor)
                except ValueError as e:
                    print(f"Skipping tensor {tensor_name} due to error: {e}")

# Save the weights and biases
save_quantized_weights_and_biases(quantized_tflite_model, "quantized_model")


Epoch 1/20
422/422 [==============================] - 4s 7ms/step - loss: 0.8676 - accuracy: 0.7275 - val_loss: 0.4597 - val_accuracy: 0.8978
Epoch 2/20
422/422 [==============================] - 2s 4ms/step - loss: 0.3199 - accuracy: 0.9098 - val_loss: 0.2019 - val_accuracy: 0.9423
Epoch 3/20
422/422 [==============================] - 2s 4ms/step - loss: 0.2342 - accuracy: 0.9321 - val_loss: 0.1650 - val_accuracy: 0.9552
Epoch 4/20
422/422 [==============================] - 2s 4ms/step - loss: 0.1987 - accuracy: 0.9422 - val_loss: 0.1489 - val_accuracy: 0.9582
Epoch 5/20
422/422 [==============================] - 2s 4ms/step - loss: 0.1748 - accuracy: 0.9488 - val_loss: 0.1349 - val_accuracy: 0.9617
Epoch 6/20
422/422 [==============================] - 2s 4ms/step - loss: 0.1613 - accuracy: 0.9517 - val_loss: 0.1379 - val_accuracy: 0.9583
Epoch 7/20
422/422 [==============================] - 3s 6ms/step - loss: 0.1464 - accuracy: 0.9566 - val_loss: 0.1257 - val_accuracy: 0.9643
Epoch 

Tensor length is: 10
Tensor length is: 10
Tensor length is: 10
Tensor length is: 10
Tensor length is: 20
Tensor length is: 20
Tensor length is: 50
Tensor length is: 50
Skipping tensor sequential_2/dense/MatMul;sequential_2/dense/Relu;sequential_2/dense/BiasAdd due to error: Tensor data is null. Run allocate_tensors() first
Skipping tensor sequential_2/dense_1/MatMul;sequential_2/dense_1/Relu;sequential_2/dense_1/BiasAdd due to error: Tensor data is null. Run allocate_tensors() first
Skipping tensor sequential_2/dense_2/MatMul;sequential_2/dense_2/Relu;sequential_2/dense_2/BiasAdd due to error: Tensor data is null. Run allocate_tensors() first


In [ ]:

weights_count = {
    'dense': 0,
    'dense_1': 0,
    'dense_2': 0,
    'dense_3': 0
}
biases_count = {
    'dense': 0,
    'dense_1': 0,
    'dense_2': 0,
    'dense_3': 0
}


current_layer = None
current_type = None

for line in lines:
    if line.startswith('#'):
        if 'weights' in line:
            current_layer = line.split('corresponds to layer: ')[1].split(' weights')[0]
            current_type = 'weights'
        elif 'biases' in line:
            current_layer = line.split('corresponds to layer: ')[1].split(' biases')[0]
            current_type = 'biases'
    else:
        if current_layer and current_type:
            values = line.strip().split()
            num_values = len(values)
            if current_type == 'weights':
                weights_count[current_layer] += num_values
            elif current_type == 'biases':
                biases_count[current_layer] += num_values

# Print the counts
print("Weights count:", weights_count)
print("Biases count:", biases_count)

# Verify counts
expected_weights_count = {
    'dense': 784 * 50,
    'dense_1': 50 * 20,
    'dense_2': 20 * 10,
    'dense_3': 10 * 10
}
expected_biases_count = {
    'dense': 50,
    'dense_1': 20,
    'dense_2': 10,
    'dense_3': 10
}

print("Expected weights count:", expected_weights_count)
print("Expected biases count:", expected_biases_count)


NameError: name 'lines' is not defined

In [ ]:
def print_scaling_factors(tflite_model):
    interpreter = tf.lite.Interpreter(model_content=tflite_model)
    interpreter.allocate_tensors()

    tensor_details = interpreter.get_tensor_details()

    for tensor_detail in tensor_details:
        tensor_name = tensor_detail['name']
        quant_params = tensor_detail['quantization_parameters']
        scale = quant_params['scales']
        zero_point = quant_params['zero_points']
        if scale.size > 0 and zero_point.size > 0:
            print(f"Tensor name: {tensor_name}")
            print(f"  Scale: {scale}")
            print(f"  Zero point: {zero_point}")

# Print the scaling factors
print_scaling_factors(quantized_tflite_model)

Tensor name: inputs
  Scale: [0.00392157]
  Zero point: [-128]
Tensor name: sequential_10/dense_3/BiasAdd/ReadVariableOp/resource
  Scale: [0.00125457]
  Zero point: [0]
Tensor name: sequential_10/dense_3/MatMul
  Scale: [0.0096693]
  Zero point: [0]
Tensor name: sequential_10/dense_2/BiasAdd/ReadVariableOp/resource
  Scale: [0.00084419]
  Zero point: [0]
Tensor name: sequential_10/dense_2/MatMul
  Scale: [0.00900361]
  Zero point: [0]
Tensor name: sequential_10/dense_1/BiasAdd/ReadVariableOp/resource
  Scale: [0.00038594]
  Zero point: [0]
Tensor name: sequential_10/dense_1/MatMul
  Scale: [0.00877394]
  Zero point: [0]
Tensor name: sequential_10/dense/BiasAdd/ReadVariableOp/resource
  Scale: [2.9944636e-05]
  Zero point: [0]
Tensor name: sequential_10/dense/MatMul
  Scale: [0.00763588]
  Zero point: [0]
Tensor name: sequential_10/dense/MatMul;sequential_10/dense/Relu;sequential_10/dense/BiasAdd
  Scale: [0.04398748]
  Zero point: [-128]
Tensor name: sequential_10/dense_1/MatMul;seque

In [ ]:
import numpy as np

class SimpleNeuron:
    def __init__(self, weights, bias):
        self.weights = np.array(weights, dtype=int)
        self.bias = int(bias)

    def feedforward(self, inputs):
        total = np.dot(self.weights, inputs) + self.bias
        return self.activation_function_ReLU(total)

    def activation_function_ReLU(self, x):
        return max(0, x)

class NeuralNetwork:
    def __init__(self, layer_config, weights, biases):
        self.layers = []
        for i in range(len(layer_config) - 1):
            layer = []
            for j in range(layer_config[i + 1]):
                neuron = SimpleNeuron(weights[i][j], biases[i][j])
                layer.append(neuron)
            self.layers.append(layer)

    def feedforward(self, inputs):
        for layer_index, layer in enumerate(self.layers):
            outputs = []
            print(f"Layer {layer_index + 1}:")
            for neuron_index, neuron in enumerate(layer):
                output = neuron.feedforward(inputs)
                outputs.append(output)
                print(f"  Neuron {neuron_index + 1}: {output}")
            inputs = outputs
        return inputs
        # for layer in self.layers:
        #     outputs = []
        #     for neuron in layer:
        #         output = neuron.feedforward(inputs)
        #         outputs.append(output)
        #     inputs = outputs  # The output of the current layer is the input to the next layer
        # return inputs

def parse_weights_biases(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    weights = []
    biases = []

    current_weights = []
    current_biases = []
    is_weight = False
    is_bias = False

    for line in lines:
        if 'weights' in line:
            if current_weights:
                weights.append(current_weights)
                current_weights = []
            is_weight = True
            is_bias = False
        elif 'biases' in line:
            if current_biases:
                biases.append(current_biases)
                current_biases = []
            is_weight = False
            is_bias = True
        else:
            if is_weight:
                current_weights.append(list(map(int, map(float, line.strip().split()))))
            elif is_bias:
                current_biases.append(int(float(line.strip())))

    if current_weights:
        weights.append(current_weights)
    if current_biases:
        biases.append(current_biases)

    return weights, biases

def load_inputs(input_file):
    data = np.loadtxt(input_file, dtype=int)
    if data.size != 784:
        raise ValueError("Each input sample must have 784 features")
    return data.reshape(784)

file_path = 'quantized_model_quantized_params.txt'
weights, biases = parse_weights_biases(file_path)

# Reorder the weights and biases based on the layer configuration
weights = [np.array(weights[3]), np.array(weights[2]), np.array(weights[1]), np.array(weights[0])]
biases = [np.array(biases[3]), np.array(biases[2]), np.array(biases[1]), np.array(biases[0])]

# Verify shapes and data types
for i, (w, b) in enumerate(zip(weights, biases)):
    print(f"Layer {i} - Weights shape: {w.shape}, Biases shape: {b.shape}, Data types: {w.dtype}, {b.dtype}")


input_file = 'mnist (3).txt'
inputs = load_inputs(input_file)

layer_config = [784, 50, 20, 10, 10]

nn = NeuralNetwork(layer_config, weights, biases)

output = nn.feedforward(inputs)
formatted_output = [f"{x:,}" for x in output]
print(f"Output: {formatted_output}")


Layer 0 - Weights shape: (50, 784), Biases shape: (50,), Data types: int64, int64
Layer 1 - Weights shape: (20, 50), Biases shape: (20,), Data types: int64, int64
Layer 2 - Weights shape: (10, 20), Biases shape: (10,), Data types: int64, int64
Layer 3 - Weights shape: (10, 10), Biases shape: (10,), Data types: int64, int64
Layer 1:
  Neuron 1: 0
  Neuron 2: 0
  Neuron 3: 0
  Neuron 4: 1910
  Neuron 5: 0
  Neuron 6: 0
  Neuron 7: 1589
  Neuron 8: 0
  Neuron 9: 605
  Neuron 10: 0
  Neuron 11: 0
  Neuron 12: 0
  Neuron 13: 590
  Neuron 14: 0
  Neuron 15: 849
  Neuron 16: 0
  Neuron 17: 0
  Neuron 18: 1321
  Neuron 19: 1828
  Neuron 20: 0
  Neuron 21: 0
  Neuron 22: 1994
  Neuron 23: 0
  Neuron 24: 252
  Neuron 25: 0
  Neuron 26: 1275
  Neuron 27: 0
  Neuron 28: 249
  Neuron 29: 1054
  Neuron 30: 0
  Neuron 31: 0
  Neuron 32: 0
  Neuron 33: 0
  Neuron 34: 0
  Neuron 35: 0
  Neuron 36: 254
  Neuron 37: 425
  Neuron 38: 0
  Neuron 39: 1186
  Neuron 40: 0
  Neuron 41: 107
  Neuron 42: 0
  Neu

<ipython-input-9-104ea4ab2e76>:82: DeprecationWarning: loadtxt(): Parsing an integer via a float is deprecated.  To avoid this warning, you can:
    * make sure the original data is stored as integers.
    * use the `converters=` keyword argument.  If you only use
      NumPy 1.23 or later, `converters=float` will normally work.
    * Use `np.loadtxt(...).astype(np.int64)` parsing the file as
      floating point and then convert it.  (On all NumPy versions.)
  (Deprecated NumPy 1.23)
  data = np.loadtxt(input_file, dtype=int)


In [ ]:
import numpy as np

def parse_weights_biases(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    weights = []
    biases = []

    current_weights = []
    current_biases = []
    is_weight = False
    is_bias = False

    for line in lines:
        if 'weights' in line:
            if current_weights:
                weights.append(current_weights)
                current_weights = []
            is_weight = True
            is_bias = False
        elif 'biases' in line:
            if current_biases:
                biases.append(current_biases)
                current_biases = []
            is_weight = False
            is_bias = True
        else:
            if is_weight:
                current_weights.append(list(map(int, map(float, line.strip().split()))))
            elif is_bias:
                current_biases.append(list(map(int, map(float, line.strip().split()))))

    if current_weights:
        weights.append(current_weights)
    if current_biases:
        biases.append(current_biases)

    return weights, biases


file_path = 'quantized_model_quantized_params (2).txt'
weights, biases = parse_weights_biases(file_path)

# Reorder the weights and biases based on the layer configuration
weights = [np.array(weights[3]), np.array(weights[2]), np.array(weights[1]), np.array(weights[0])]
biases = [np.array(biases[3]), np.array(biases[2]), np.array(biases[1]), np.array(biases[0])]

layer_config = [784, 50, 20, 10, 10]

# Format weights into a list of 2D arrays
weights_3d = []
for i in range(len(layer_config) - 1):
    layer_weights = np.zeros((layer_config[i + 1], layer_config[i]), dtype=int)
    for j in range(layer_config[i + 1]):
        layer_weights[j] = weights[i][j]
    weights_3d.append(layer_weights)

# Format biases into a list of 1D arrays
biases_2d = []
for i in range(len(layer_config) - 1):
    layer_biases = np.zeros((layer_config[i + 1], 1), dtype=int)
    for j in range(layer_config[i + 1]):
        layer_biases[j] = biases[i][j]
    biases_2d.append(layer_biases)

output_file = 'formatted_weights_biases.txt'

with open(output_file, 'w') as f:
    f.write("parameter int WEIGHTS [][][] = '{\n")
    for layer_weights in weights_3d:
        f.write("    '{\n")
        for neuron_weights in layer_weights:
            f.write("        '{" + ', '.join(map(str, neuron_weights)) + "},\n")
        f.write("    },\n")
    f.write("};\n\n")

    f.write("parameter int BIASES [][] = '{\n")
    for layer_biases in biases_2d:
        f.write("    '{" + ', '.join(map(str, layer_biases.flatten())) + "},\n")
    f.write("};\n")

print(f"Formatted weights and biases have been written to {output_file}")


Formatted weights and biases have been written to formatted_weights_biases.txt
